## **COMO RODAR?**

Dentro de cada célula, apertar *Ctrl+Enter* ou clicar em *Run* - ícone superior da página.

**Observação 1:** 
- Rodar as células em ordem. A última célula terá o resultado que deseja. 

Caso queira rodar todas as células de uma vez: cell -> Run All

**Observação 2:** 
- Qualquer modificação em uma célula, todas devem ser rodadas novamente e em ordem. 

# Observação 

## Por que não houve divisão em classes neste código?

A *orientação objeto* no Jupyter é complicada e, como o notebook está sendo utilizando apenas para exemplificação do processamento de linguagem natural, optou- se por deixar os comentários em cima de cada processo do programa. 

### Instalações necessárias antes de utilizar este módulo. 

    ! pip install pattern
    ! pip install pyinflect
    ! pip install deep_translator
    

In [15]:
import nltk
import pyinflect
import spacy 
from deep_translator import GoogleTranslator

In [16]:
nltk.download()
spacy.cli.download("en_core_web_sm")

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Mudança da voz verbal em uma sentença. 

        1) Traduzir para o inglês uma vez que há funções mias otmizadas no NLTK   para essa linguagem.
        2) Tokenização
        3) Verificar o tempo verbal
        4) Realizar a modificação 
        5) Traduzir para o português

----------
### Funções responsáveis por transformar a voz

As funções serão chamadas a realização da tokenixação do texto em inglês

In [17]:
def modificar_para_ativa (objeto, sujeito, tagged, posicao_verbo, posicao_objeto,
                  posicao_sujeito, verbo_passado):

    novo_texto = []

    #Responsável por realizar as trocas
    for i in range (len(tagged)):
        if i == posicao_verbo:
            novo_texto.append(verbo_passado)
        elif i == posicao_sujeito:
            novo_texto.append(objeto)
        elif i == posicao_objeto:
            novo_texto.append(sujeito)
        else:
            novo_texto.append(tagged[i][0])


    #Remove verbo no participio passado
    novo_texto.pop(posicao_verbo+1)

    # Excluir o by pois essa preposição não existe na voz ativa
    novo_texto.remove('by')

    return novo_texto


In [18]:
def modificar_para_passiva (objeto, sujeito, tagged, posicao_verbo, posicao_objeto,
              posicao_sujeito, verbo_participio_passado):

    novo_texo = []
    for tag in tagged:
        if tag[0] == objeto:
            tag_objeto = tag[1]
    for i in range (len(tagged)):

        if i == posicao_verbo:
            # Valida qual conjugação do verbo to be no passado se adequa ao objeto
            # O objeto será o novo sujeito
            if ((tag_objeto == 'NNS') or 
                (tag_objeto in ['they', 'They', 'we', 'We', 'you', 'You'])):
                    novo_texo.append('were')
            else:
                    novo_texo.append('was')
            novo_texo.append(verbo_participio_passado)
            
        elif i == posicao_sujeito:
            novo_texo.append(objeto)
        elif i == posicao_objeto:
            #Adicionar a preposição by
            novo_texo.insert(i,'by')
            novo_texo.append(sujeito)
        else:
            novo_texo.append(tagged[i][0])

    return novo_texo


-----------
##                                           Início do processo de mudança de voz

### **Tradução da sentença e tratamento dela**

In [19]:
sentenca = "O carro atropelou o menino"

if '.' in sentenca:
    sentenca = sentenca.replace('.', '')

traduzir_ingles  = GoogleTranslator(source='portuguese', target='english').translate(sentenca)
print(f'Tradução para o inglês: {traduzir_ingles}')

Tradução para o inglês: The car hit the boy



**Verificação se há um trecho entre vírgulas separando sujeito e o verbo**


In [20]:
# Caso não exista um trecho entre vírgulas na sentença
palavra_antes_verbo = None 

tokens = nltk.word_tokenize(traduzir_ingles)
tagged = nltk.pos_tag(tokens) 
for i,tag in enumerate(tagged):
    if tag[0] == ',':
        palavra_antes_verbo = tagged[i-1][0]
        break
    
# Armazenar em uma string o trecho entre vírgula
# Retirar esse trecho da sentenca em inglês por enquanto
if palavra_antes_verbo != None:
    for i in range(len(traduzir_ingles)):
        if traduzir_ingles[i] == ',':
            posicao_virgula1 = i
            break

    for i in range (posicao_virgula1+1, len(traduzir_ingles)):
        if traduzir_ingles[i] == ',':
            posicao_virgula2 = i
            break

    entre_virgulas = traduzir_ingles[posicao_virgula1:posicao_virgula2+1]
    traduzir_ingles = traduzir_ingles.replace(entre_virgulas,"")
    print(f'Tradução sem o trecho entre vírgulas: {traduzir_ingles}')

-----------
### **Tokenização**

In [21]:
tokens = nltk.word_tokenize(traduzir_ingles)
#Lista de tuplas cujos elementos são respectivamente a palavra e sua morfologia
tagged = nltk.pos_tag(tokens) 
display(tagged)

[('The', 'DT'), ('car', 'NN'), ('hit', 'VBD'), ('the', 'DT'), ('boy', 'NN')]

----------
### Identificar o sujeito e o objeto + Verificação se a voz é passiva ou ativa 

In [22]:
# Encontra o objeto da frase
for i in range(len(tagged)-1, -1, -1):
    if (tagged[i][1] == 'NN') or (tagged[i][1] == 'NNS'):
        objeto = tagged[i][0]
        posicao_objeto = i
        break
        
# Encontra o sujeito da frase
for i,tag in enumerate(tagged):
    if (tag[1] == 'NN') or (tag[1] == 'NNS'):
        sujeito = tag[0]
        posicao_sujeito = i
        break
        
#Encontra a posição do verbo        
for i in range(len(tagged)):
    #Verifica se o token é um verbo (independente do tempo)
    if tagged[i][1][0] == 'V':
        posicao_verbo = i
        break

#Necessidade de tranformar o texto em um objeto spacy.load para trabalhar com tempos verbais
nlp = spacy.load("en_core_web_sm")
doc_dep = nlp(traduzir_ingles)

**Verifica o tipo de voz a frase atual está empregada**

In [23]:
verbo_passado = None
verbo_participio_passado = None

# Verifica se a voz é passiva (verb to be in the pass + verb in the past participle)
if (tagged[posicao_verbo][1] == 'VBD') and (tagged[posicao_verbo+1][1] == 'VBN'):
    for i in range(1,len(doc_dep)):
        token = doc_dep[i-1]
        proximo_token = doc_dep[i]
        if (token.tag_ == 'VBD')  and (proximo_token.tag_ == 'VBN'): 
            verbo_passado = proximo_token._.inflect("VBD")

    #Representa uma lista com a sentenca ordenada para outra voz
    traducao_para_portugues = modificar_para_ativa(objeto, sujeito,tagged, posicao_verbo, 
                                     posicao_objeto, 
                                     posicao_sujeito, verbo_passado)


# Senão a voz é ativa (verb in the past)
else:
    for i in range(len(doc_dep)):
        token = doc_dep[i]      
        if token.tag_ == 'VBD': 
            verbo_participio_passado = token._.inflect("VBN")

    #Representa uma lista com a sentenca ordenada para outra voz
    traducao_para_portugues = modificar_para_passiva(objeto, sujeito,tagged, posicao_verbo, 
                                     posicao_objeto, 
                                     posicao_sujeito, verbo_participio_passado)

display(f'Verbo no passado: {verbo_passado}')
display(f'Verbo no participio passado: {verbo_participio_passado}')
print(traducao_para_portugues)

'Verbo no passado: None'

'Verbo no participio passado: hit'

['The', 'boy', 'was', 'hit', 'by', 'the', 'car']


----------
### Tranformar a lista de palavras em uma string

A partir de agora, caso existisse vírgula entre o sujeito e o verbo, a string é retificada. 

In [24]:
if palavra_antes_verbo != None:
    #Uma lista com todos os elementos da string
    entre_virgulas = entre_virgulas.split()

    indice_palavra_antes = traducao_para_portugues.index(palavra_antes_verbo)
    
    for i in range(len(entre_virgulas)):
        traducao_para_portugues.insert(indice_palavra_antes+1+i, entre_virgulas[i])

    #Retirar a segunda vírgula caso entre_virgulas estiver no final da frase na modificação.

    if entre_virgulas[-1] == traducao_para_portugues[-1]:
        ultimo_elemento = entre_virgulas[-1]
        ultimo_elemento = ultimo_elemento.replace(',', "")
        traducao_para_portugues[-1] = ultimo_elemento
    else:
        pass

    traducao_para_portugues = " ".join(traducao_para_portugues)

else:
    traducao_para_portugues = " ".join(traducao_para_portugues)

traducao_para_portugues 

'The boy was hit by the car'

------------
### Retorno para o português

In [25]:
traducao_para_portugues = traducao_para_portugues + '.'
nova_sentenca = GoogleTranslator(source='english', target='portuguese').translate(traducao_para_portugues)
sentenca = sentenca + '.'
print(f'Sentença para ser modificada: {sentenca}')
print(f'Sentença modificada: {nova_sentenca}')

Sentença para ser modificada: O carro atropelou o menino.
Sentença modificada: O menino foi atropelado pelo carro.
